In [2]:
import cv2
import os
import numpy as np
# change file location according to your drive
weight_path=os.getcwd()+'\\yolov3_face_training_3000.weights' #weight file path
cfg_path=os.getcwd()+'\\yolov3_face_testing.cfg' # Configuration file path 
classes = ['f','b'] #  front side and back side
threshold=0.6
font = cv2.FONT_HERSHEY_PLAIN
font_=cv2.FONT_HERSHEY_COMPLEX
net = cv2.dnn.readNet(weight_path,cfg_path)

#### overlay space
cap = cv2.VideoCapture('C:\\Users\\Vishal\\AIS_vision100.ai\\crowd_detection\\video_crowd_4.mp4')
while True:
    _, img = cap.read()
    height, width, _ = img.shape

    blob = cv2.dnn.blobFromImage(img, 1/255, (416, 416), (1,0,0), swapRB=True, crop=False)
    net.setInput(blob)
    output_layers_names = net.getUnconnectedOutLayersNames()
    layerOutputs = net.forward(output_layers_names)

    boxes = []
    confidences = []
    class_ids = []

    for output in layerOutputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > threshold:
                center_x = int(detection[0]*width)
                center_y = int(detection[1]*height)
                h= int(detection[2]*width)
                w= int(detection[3]*height*0.8)

                x = int(center_x - w/2)
                y = int(center_y - h/2)

                boxes.append([x, y, w, h])
                confidences.append((float(confidence)))
                class_ids.append(class_id)
                    
    
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    if len(indexes)>0:
        for i in indexes.flatten():
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = str(round(confidences[i],2))
            ############

            if label==classes[0]:
                cv2.rectangle(img, (x,y), (x+h,y+w), (255,200,200), 2)
                
            if label==classes[1]:
                cv2.rectangle(img, (x,y), (x+h,y+w), (200,255,200), 2)
                
    # label AIS
    text = str('AIS Solutions Pvt. Ltd.')
    # get the width and height of the text box
    (text_width, text_height) = cv2.getTextSize(text, font, fontScale=1, thickness=1)[0]
    # set the text start position
    text_offset_x =250
    text_offset_y =330
    # make the coords of the box with a small padding of two pixels
    box_coords = ((text_offset_x, text_offset_y+5), (text_offset_x + text_width + 10,text_offset_y - text_height-5))
    cv2.rectangle(img, box_coords[0], box_coords[1], (0,250,255), cv2.FILLED)
    cv2.putText(img, text, (text_offset_x, text_offset_y), font,fontScale=1, color=(0,0,0), thickness=1)

                
    # people count text edit          
    text = str('People Count : ')+str(len(indexes))
    # get the width and height of the text box
    (text_width, text_height) = cv2.getTextSize(text, font_, fontScale=1, thickness=1)[0]
    # set the text start position
    text_offset_x =180
    text_offset_y =40
    # make the coords of the box with a small padding of two pixels
    box_coords = ((text_offset_x, text_offset_y+20), (text_offset_x + text_width + 10, text_offset_y - text_height-20))
    cv2.rectangle(img, box_coords[0], box_coords[1], (0,0,0), cv2.FILLED)
    cv2.putText(img, text, (text_offset_x, text_offset_y), font_, fontScale=1, color=(200,200,0), thickness=2)

    
    cv2.imshow('Image', img)
    key=cv2.waitKey(1)
    if key ==ord('v'): 
        break 
cap.release()
cv2.destroyAllWindows()